In [7]:
import murraylab_tools.echo as mt_echo
import os.path

# Echo Setup - Salt Calibration Example

This notebook shows an example of how to programmatically create a picklist for an experiment using the murraylab_tools's Echo subpackage. The experiment in question is a TX-TL buffer calibration experiment. Whenever we make a fresh batch of TX-TL, we make a new buffer solution with custom MgCl and KCl concentrations optimized for maximum production from a GFP plasmid. To find those concentrations, we'll have to run TX-TL reactions with a range of MgCl and KCl concentrations. 

First, we choose where to put our files (in this case, in a "salt calibration" folder).

In [8]:
salt_calibration_input_folder = os.path.join("salt_calibration","inputs")
salt_calibration_output_folder = os.path.join("salt_calibration","outputs")
plate_file = os.path.join(salt_calibration_output_folder, "salt_calibration_plate.dat")
output_name = os.path.join(salt_calibration_output_folder, "salt_calibration_experiment")

Next, we define input materials. Even though this is a TX-TL experiment, we can't use a standard `TXTLMasterMix` class to define a master mix, because there's no buffer solution for this extract yet -- we'll need to add each buffer component to the master mix, leaving out only MgCl and KCl salts. We'll use a `MixtureMaterial` object to represent this master mix.

In [9]:
# Define master mix:
txtl            = mt_echo.EchoSourceMaterial("TX-TL extract", 1, units = "x") # x
PEG             = mt_echo.EchoSourceMaterial("PEG-8000", 1, units = "x") # x
energy_solution = mt_echo.EchoSourceMaterial("Energy Solution", 1, units = "x") # x
amino_acids     = mt_echo.EchoSourceMaterial("Amino Acids", 6, units = "mM") # mM
water           = mt_echo.EchoSourceMaterial("Water", 1, units = "x") # x
gfp_plasmid     = mt_echo.EchoSourceMaterial("pBEST-GFP", 9, units = "nM") # nM

master_mix = mt_echo.MixtureMaterial(name = "Master Mix")

master_mix.add_material(txtl, 0.33)
master_mix.add_material(PEG, .1)
master_mix.add_material(energy_solution, .2)
master_mix.add_material(amino_acids, 1)
master_mix.add_material(gfp_plasmid, 1)

print("Master mix contains:")
for material, concentration, dimensionality in master_mix.materials:
    print(f"\t{material} at concentration {concentration}, dimensionality = {dimensionality}")
print("fill material: " + str(master_mix.fill_material))
# print(master_mix.text_recipe())

Master mix contains:
	TX-TL extract (1.000 x) at concentration 0.33, dimensionality = final_concentration
	PEG-8000 (1.000 x) at concentration 0.1, dimensionality = final_concentration
	Energy Solution (1.000 x) at concentration 0.2, dimensionality = final_concentration
	Amino Acids (6.000 mM) at concentration 1, dimensionality = final_concentration
	pBEST-GFP (9.000 nM) at concentration 1, dimensionality = final_concentration
fill material: None


We'll also need to add salts.

In [10]:
# Final concentrations
mg_concs = [3, 5, 7, 9, 11]
k_concs = [60, 80, 100, 120, 140]

# Salt materials
mg_salt = mt_echo.EchoSourceMaterial("Mg", 200, units = "mM")
k_salt = mt_echo.EchoSourceMaterial("K", 6000, units = "mM")

Finally, we create an `EchoRun` object, and add our materials. Each well should contain:
* Master mix, including buffer components.
* MgCl at some concentration.
* KCl at some concentration.
* Water to fill to 10 uL. 

In [11]:
# Build an EchoRun object
txtl_plate = mt_echo.SourcePlate(filename = plate_file)
txtl_echo_calculator = mt_echo.EchoRun(plate = txtl_plate)

txtl_echo_calculator.dilution_series(mg_salt, k_salt, mg_concs, k_concs, "A1")
txtl_echo_calculator.add_material_to_all(master_mix) # If you try to fill this, it breaks
txtl_echo_calculator.fill_all_wells_with(water)

txtl_echo_calculator.write_picklist(output_name)

This will produce a picklist in "salt_calibration/salt_calibration_experiment_EchoInput" and a human-readable instruction file in "salt_calibration/salt_calibration_experiment_experiment_overview.txt". Here are the human-readable instructions:

In [12]:
salt_filename = os.path.join("salt_calibration","outputs", "salt_calibration_experiment_experiment_overview.txt")
with open(salt_filename,'r') as salt_file:
    for line in salt_file:
        print(line, end = "")

Materials used:

Mg:
	stock concentration: 200.00 mM
	total volume: 29.75 uL
K:
	stock concentration: 6000.00 mM
	total volume: 25.12 uL
Master Mix:
	Stock Concentration: 1 
	Total Volume: 352875.00
		128.28 uL TX-TL extract (1.000 x)
		38.87 uL PEG-8000 (1.000 x)
		77.74 uL Energy Solution (1.000 x)
		64.79 uL Amino Acids (6.000 mM)
		43.19 uL pBEST-GFP (9.000 nM)
Water:
	stock concentration: 1.00 x
	total volume: 31.25 uL

On the source plate:
	Add 29.75 uL of Mg in well: A21
	Add 25.125 uL of K in well: A23
	Add 65.0 uL of Master Mix in wells: B09, B10, B11, B12, B13
	Add 27.875 uL of Master Mix in well: B14
	Add 31.25 uL of Water in well: B16